In [ ]:
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)
import os 
import plotly.graph_objects as go
import pandas as pd
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

from utils.afrr_preprocessing import preprocess_afrr_data
from afrr_price_ts_forecast.ensamble_forecast.lr_forecaster import run_lr_pipeline
from afrr_price_ts_forecast.ensamble_forecast.gp_forecaster import run_gp_pipeline
from afrr_price_ts_forecast.ensamble_forecast.xgb_forecaster import run_xgb_pipeline


from utils.forecast_utils import get_forecast_params
from darts.timeseries import concatenate

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
data_path = "/home/alqua/git/boiler_OptiBid/data/afrr_price.parquet"
train_start = "2024-10-01 22:00:00"
test_start = "2025-01-09 22:00:00"
test_end = "2025-03-20 22:00:00"

(afrr_pr_ts_scl_train, 
afrr_pr_ts_scl_test, 
afrr_pr_ts_orig_train, 
afrr_pr_ts_orig_test, 
exog_ts_scl_train, 
exog_ts_scl_test,
afrr_pr_scaler
) = preprocess_afrr_data(data_path, train_start, test_start, test_end)

----------------------------------------
EMBEDDING SUMMARY:
Embedding dimension	:  120
Trajectory dimensions	: (120, 3672)
Complete dimension	: (120, 3672)
Missing dimension     	: (120, 0)
----------------------------------------
DECOMPOSITION SUMMARY:
Rank of trajectory		: 120
Dimension of projection space	: 114
Characteristic of projection	: 0.9997


In [9]:
lr_results = pd.read_parquet("linear_models.par")
gp_results = pd.read_parquet("gaussian_process.par")
xgboost_results = pd.read_parquet("xgboost_model_forecast.par")



In [11]:
afrr_pr_ts_orig = concatenate([afrr_pr_ts_orig_train, afrr_pr_ts_orig_test], axis=0)
afrr_pr_ts_orig = afrr_pr_ts_orig.with_columns_renamed(['aFRR_UpCapPriceEUR'], col_names_new=["afrr_up_cap_price"])
result_df = lr_results.join([xgboost_results, gp_results, afrr_pr_ts_orig.pd_dataframe()])

In [22]:


test_start_dt = pd.to_datetime("2025-01-09 22:00:00")

fig = go.Figure()
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
fig.update_layout(height=400)

# Add traces with the data
fig.add_trace(go.Scatter(name=r'AFRR Up Cap Price', 
                        x=result_df.index, 
                        y=result_df['afrr_up_cap_price'],
                        showlegend=True
                        ))
fig.add_trace(go.Scatter(name=r'AFRR Up Cap Price Linear', 
                        x=result_df.index, 
                        y=result_df['gp_afrr_up_cap_price_0.5'],
                        showlegend=True))
fig.add_trace(go.Scatter(name=r'AFRR Up Cap Price GP', 
                        x=result_df.index, 
                        y=result_df['lr_afrr_up_cap_price_0.5'],
                        showlegend=True,
                        line=dict(color='green')))
fig.add_trace(go.Scatter(name=r'AFRR Up Cap Price XGBoost', 
                        x=result_df.index, 
                        y=result_df['xgb_afrr_up_cap_price'],
                        showlegend=True, 
                        line=dict(color='violet')))

fig.add_shape(
    type="line",
    x0=test_start_dt,
    x1=test_start_dt,
    y0=0,
    y1=1,
    yref="paper", 
    line=dict(color="red", width=2, dash="dash")
)

fig.add_annotation(
    x=test_start_dt,
    y=1,
    yref="paper",
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40
)

fig.show()
